In [1]:
#Import
import time
import mysql.connector
import os
import requests as r
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [27]:
def dev_finder(zones):
    start= time.time()
    val_list = []
    validator = open("validation.txt", "r")
    for line in validator:
        val_list.append(line.rstrip())

    mydb = mysql.connector.connect(
      host=val_list[0],
      user=val_list[1],
      passwd=val_list[2],
      database=val_list[3],
      port=val_list[4]
    )
    quart_dict = {}
    for i in zones:
        mycursor = mydb.cursor()
        sql = """SELECT COUNT(*) as nr FROM team12dublinbus.final where zone = """ +str(1)+""";"""
        mycursor.execute(sql)
        result_list = []
        rand_no = 0
        for (r) in mycursor:
            rand_no = str(round(10000/int(r[0]), 4))
            
        print("Received number interval from DB for zone {0}".format(i))
        print("Receiving dataframe for zone {0}".format(i))
        
        mycursor = mydb.cursor()
        sql = """SELECT travel_time
        FROM team12dublinbus.final
        WHERE zone = """+str(i)+""" and RAND() < """+str(rand_no)+""";"""
        mycursor.execute(sql)
        
        print("Received dataframe for zone {0}. Processing frame...".format(i))
        result_list = []
        for (l) in mycursor:
            result_list.extend([l])
        
        frame = DataFrame.from_records(result_list)
        frame.columns = ['time_of_day']
        
        quart_dict["zone_{0}".format(i)] = [frame.time_of_day.quantile([0.15,0.5,0.85]).values[0],frame.time_of_day.quantile([0.15,0.5,0.85]).values[1],frame.time_of_day.quantile([0.15,0.5,0.85]).values[2]]
    return(quart_dict)
        

In [28]:
result = dev_finder(["1","2","3","4","5","6","7","8","9","10","11"])

Received number interval from DB for zone 1
Receiving dataframe for zone 1
Received dataframe for zone 1. Processing frame...
Received number interval from DB for zone 2
Receiving dataframe for zone 2
Received dataframe for zone 2. Processing frame...
Received number interval from DB for zone 3
Receiving dataframe for zone 3
Received dataframe for zone 3. Processing frame...
Received number interval from DB for zone 4
Receiving dataframe for zone 4
Received dataframe for zone 4. Processing frame...
Received number interval from DB for zone 5
Receiving dataframe for zone 5
Received dataframe for zone 5. Processing frame...
Received number interval from DB for zone 6
Receiving dataframe for zone 6
Received dataframe for zone 6. Processing frame...
Received number interval from DB for zone 7
Receiving dataframe for zone 7
Received dataframe for zone 7. Processing frame...
Received number interval from DB for zone 8
Receiving dataframe for zone 8
Received dataframe for zone 8. Processing f

In [17]:
result

{'zone_12': [19.0, 27.0]}

In [5]:
with open('1-11.pickle', 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:


with open('1-11.pickle', 'rb') as handle:
    dict = pickle.load(handle)

In [ ]:
'zone_12': [19.0, 27.0]

In [19]:
dict

{'zone_1': [70.0, 284.0],
 'zone_2': [32.0, 112.0],
 'zone_3': [21.0, 75.0],
 'zone_4': [15.0, 58.0],
 'zone_5': [8.0, 95.0],
 'zone_6': [65.19999999999999, 70.1],
 'zone_7': [7.6, 11.4],
 'zone_8': [5.0, 94.0],
 'zone_9': [81.75, 104.65],
 'zone_10': [1.0, 98.55000000000001],
 'zone_11': [51.0, 75.6]}

In [20]:
dict["zone_12"] = [19.0, 27.0]

In [21]:
dict

{'zone_1': [70.0, 284.0],
 'zone_2': [32.0, 112.0],
 'zone_3': [21.0, 75.0],
 'zone_4': [15.0, 58.0],
 'zone_5': [8.0, 95.0],
 'zone_6': [65.19999999999999, 70.1],
 'zone_7': [7.6, 11.4],
 'zone_8': [5.0, 94.0],
 'zone_9': [81.75, 104.65],
 'zone_10': [1.0, 98.55000000000001],
 'zone_11': [51.0, 75.6],
 'zone_12': [19.0, 27.0]}

In [22]:
with open('variance.pickle', 'wb') as handle:
    pickle.dump(dict, handle, protocol=pickle.HIGHEST_PROTOCOL)